In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests

***Three ways of loading a plate***
1. After hyphae have been extracted (and the network has been cleaned). Then the inst name is needed : refer yourself to `amftrack/notebooks/analysis/data_info.py` where all analysed instances are referenced. Use the `get_exp` function with arguments corresponding to the instance and the directory where you expect to find the analysed plate (most often `directory_project`). If you ask me I can also analyse a plate for you. 
2. Before hyphae are extracted but after node identification, then chose manually the dates that you want to load using get_date_datetime and selecting the right begin and end depending on which dates you need. Then create an experiment instance and load the dates using the `.load()` method.
3. If you don't care about the labelling of the nodes you can follow the same procedure but setting the labeled flag in the `.load()` method to `False`.

Method 1

In [3]:
directory = directory_project


Method 2 and 3, find the dates of interest.

In [4]:
plate_number = 102
i,date = get_begin_index(plate_number,directory)

0 102
153 69
259 None
260 102


In [5]:
plate = get_postion_number(plate_number)
begin = i+2
end = i+ 5
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
print(dates[0],dates[-1])

2020-12-26 20:04:00 2020-12-27 08:03:00


In [6]:
# exp = get_exp((9,0,11),directory)
exp = Experiment(plate,directory)
exp.load(dates) #for method 2
# exp.load(dates, labeled= False) # for method 3

***Load the skeletons for visualisation purposes***

This may take some time, go grab a coffee

In [7]:
exp.load_compressed_skel()

***Let's look at the network***

In [8]:
exp.plot_raw(3)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
nodes = [node.label for node in exp.nodes]
times = [0]
exp.plot(times,[nodes]*len(times))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plot_raw_plus(exp,0,nodes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
node = Node(113,exp)
node.show_source_image(0,1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
begin = Node(115,exp)
end = Node(110,exp)
edge = Edge(begin,end,exp)
# edge.get_length_um(0)

In [30]:
def get_length_um(edge, t):
    pixel_conversion_factor = 1.725
    length_edge = 0
    pixels = edge.pixel_list(t)
    for i in range(len(pixels) // 10 + 1):
        if i * 10 <= len(pixels) - 1:
            length_edge += np.linalg.norm(
                np.array(pixels[i * 10])
                - np.array(pixels[min((i + 1) * 10, len(pixels) - 1)])
            )
    #         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
    return length_edge * pixel_conversion_factor

In [35]:
get_length_um(edge,0)

88.0527327008531

In [36]:
edge.width(0)

5.310868505409786

In [39]:
nx.shortest_path(exp.nx_graph[0],113,100)

[113, 103, 100]